# Premières estimations sur les stations météorologiques

Dans ce notebook, nous allons effectuer une prise en main de Julia, des données ainsi que des lois d'estimation GEV (Generalized Extreme Values).

Dans l'ordre, nous allons :

1. Charger et correctement mettre en forme les données météorologiques
2. Définir les fonctions d'estimation
3. Estimer par la log-vraisemblance les paramètres pour chaque station :
    a) d'abord sans spécifier le paramètre $\xi$.
    b) puis en considérant que toutes les stations disposent du même paramètre $\xi$.
4. Vérifier les proximités des paramètres pour les stations voisines

## 0. Chargement des bibliothèques utiles et des variables globales

In [1]:
using CSV, DataFrames, Distributions, Random, StatsBase
using Extremes, Dates, Gadfly
using Optim

import Plots #pour faire des graphiques

include("functions.jl")

BIC_GEV (generic function with 1 method)

In [2]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees

6-element Vector{String}:
 "NB"
 "NL"
 "NS"
 "ON"
 "PE"
 "QC"

## 1. Chargement et mise en forme des données

In [ ]:
station_list = CSV.read("dat/_station_list.csv", DataFrame)

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

In [ ]:
dat = DataFrame(StationName = String[],
                StationID = String[],
                Year = Int64[],
                Duration = String[],
                Pcp = Float64[])

for i in 1:(nrow(station_list))
    df = load_station(station_list[i,:ID])
    df[!, :StationName] .= station_list[i,:Name]
    df[!, :StationID] .= station_list[i, :ID]
    append!(dat, df)
end

first(dat, 10)

On sélectionne ensuite les stations pour lequels il y a une duree d'enregistrement souhaitee (stocké dans la variable DURATION)

In [ ]:
DURATION = "24 h"

filter!(row -> row[:Duration] == DURATION, dat)
first(dat, 10)

## 2. Fonctions utiles pour la suite

In [ ]:
# fonction pour avoir directement
function Pcp(stationID::String)
    y = dat[dat[:,:StationID].== stationID,:Pcp]
    return y
end

On définit ensuite deux fonctions assez similaires, permettant pour un vecteur donné d'estimer les parametres de la GEV correspondante, à l'aide du package Optim.

La première fonction estime tous les paramètres, tandis que la deuxième prend en paramètre un $\xi$ fixé, pour les deux modes d'estimation différents de la suite.

In [ ]:
function GEVparameters(Y::Vector{Float64})
    function f(p::Vector{Float64})
        return -logL(Y, p[1], p[2], p[3])
    end
    
    μ₀ = mean(Y)
    σ₀ = std(Y)
    ξ₀ = 0
    p₀ = [μ₀, σ₀, ξ₀]
    p = p₀
    
    try
        res = optimize(f, p₀)
        
         
        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
        
    catch
        println("Error of scale with this vector")
    end
   
    
    return p
end

In [ ]:
function GEVparameters_xi(Y::Vector{Float64}, ξ::Real)
    function f(p::Vector{Float64})
        return -logL(Y, p[1], p[2], ξ)
    end
    
    μ₀ = mean(Y)
    σ₀ = std(Y)
    p₀ = [μ₀, σ₀]
    p = p₀
    
    try
        res = optimize(f, p₀)
        
         
        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
        
    catch
        println("Error of scale with this vector")
    end
   
    
    return p
end

## 3. Estimation

On commence avec une estimation des parametres en considérant que les xi sont indépendants.

In [ ]:
parameters = DataFrame(StationName = String[],
                        StationID = String[],
                        μₒ = Float64[],
                        ϕₒ = Float64[],
                        ξₒ = Float64[])

for i in 1:(nrow(station_list))
    y = Pcp(station_list[i, :ID])
    par_optim = GEVparameters(y)
    df = DataFrame(StationName = station_list[i, :Name],
                    StationID = station_list[i, :ID],
                    μₒ = par_optim[1],
                    ϕₒ = log(par_optim[2]),
                    ξₒ = par_optim[3])
    append!(parameters, df)
end

CSV.write("results/parameters_$DURATION.csv", parameters)
first(parameters, 10)

In [ ]:
ξ = 0 #parametre xi a ajuster

parameters_xi = DataFrame(StationName = String[],
                        StationID = String[],
                        μ = Float64[],
                        σ = Float64[])

for i in 1:(nrow(station_list))
    y = Pcp(station_list[i, :ID])
    par = GEVparameters_xi(y, ξ)
    df = DataFrame(StationName = station_list[i, :Name],
                    StationID = station_list[i, :ID],
                    μ = par[1],
                    σ = par[2])
    append!(parameters_xi, df)
end

CSV.write("results/parameters_$DURATION _$ξ.csv", parameters_xi)

first(parameters_xi, 10)

### Utilisation du package Extremes

In [ ]:
parameters_ex = DataFrame(StationName = String[],
                        StationID = String[],
                        μₑ = Float64[],
                        ϕₑ = Float64[],
                        ξₑ = Float64[],
                        BIC = Float64[])

for i in 1:(nrow(station_list))
    y = Pcp(station_list[i, :ID])
    
    μ₀ = mean(y)
    ϕ₀ = log(std(y))
    ξ₀ = 0
    p₀ = [μ₀, ϕ₀, ξ₀]
    par_ex = p₀
    BIC_var = 0
    
    try
        par_ex = gevfit(y).θ̂
        BIC_var = BIC_GEV(y)
    catch
        println("L'algorithme n'a pas convergé")
    end
    
    df = DataFrame(StationName = station_list[i, :Name],
                        StationID = station_list[i, :ID],
                        μₑ = par_ex[1],
                        ϕₑ = par_ex[2],
                        ξₑ = par_ex[3],
                        BIC = BIC_var)
    
    append!(parameters_ex, df)
end

CSV.write("results/parameters_ex_$DURATION.csv", parameters_ex)
first(parameters_ex, 10)

In [ ]:
parameters_δ = DataFrame(StationName = String[],
                        StationID = String[],
                        δμ = Float64[],
                        δϕ = Float64[],
                        δξ = Float64[])

for i in 1:(nrow(station_list))    
    df = DataFrame(StationName = station_list[i, :Name],
                        StationID = station_list[i, :ID],
                        δμ = parameters[i, :μₒ] - parameters_ex[i, :μₑ],
                        δϕ = parameters[i, :ϕₒ] - parameters_ex[i, :ϕₑ],
                        δξ = parameters[i, :ξₒ] - parameters_ex[i, :ξₑ])
    
    append!(parameters_δ, df)
end

CSV.write("results/parameters_diff_$DURATION.csv", parameters_δ)
first(parameters_δ, 10)